In [1]:
from date.date_env import DateEnv

In [2]:
env = DateEnv()

In [3]:
env.reset() # reset environment to a new, random state
print(env.render())


{'previous': {'last_action': 0, 'last_response': 0}, 'me_attributes': {'confidence': 0}, 'partner_attributes': {'interests': {'cars': 0, 'sports': 2, 'literature': 1, 'history': 3, 'machine learning and artificial intelligence': 1}, 'attraction': 2, 'romance': 0}}


In [4]:
env.reset()

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.sample_actions()
    state, reward, done = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'state': env.render(),
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 1
Penalties incurred: 0


In [14]:
reward_dict = {topic: {-1: (-1, False), 0: (0, False), 1: (1, False),  -2: (-3, True)} for topic in 
     ['cars', 'sports', 'literature', 'history', 'machine learning and artificial intelligence']}
reward_dict['flirt'] = {2: (12, True), -1: (-1, False), 0: (0, False), 1:(1, False), -2: (-3, True)}
reward_dict['leave'] = {0: (-2, True)}
reward_dict['drink'] = {0: (-1, False)}

In [15]:
env.reward_dict = reward_dict

In [16]:
import numpy as np
q_table = np.zeros([env.n_actions, env.n_responses, env.date.me.max_confidence +1, env.n_actions])

In [25]:
#%%time
"""Training the agent"""

import random
#from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100000):
    state = env.reset()

    epochs, reward, = 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.sample_actions() # Explore action space
        else:
            action = np.argmax(q_table[state[0], state[1], state[2]]) # Exploit learned values

        next_state, reward, done = env.step(action)
        
        old_value = q_table[state[0], state[1], state[2], action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state[0], state[1], state[2], action] = new_value

        
        state = next_state
        epochs += 1
        
    if i % 10000 == 0:
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 10000
Episode: 20000
Episode: 30000
Episode: 40000
Episode: 50000
Episode: 60000
Episode: 70000
Episode: 80000
Episode: 90000
Training finished.



In [26]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_successes, total_reward = 0, 0, 0
episodes = 1000

frames = []

for i in range(episodes):
    state = env.reset()
    epochs, successes, reward = 0, 0, 0
    
    done = False
    frame_set = []
    while not done:
        old_state = env.render()
        action = np.argmax(q_table[state])
        state, reward, done = env.step(action)

        if reward == 12:
            successes += 1

        # Put each rendered frame into dict for animation
        frame_set.append({
            'state': old_state,
            'action': action,
            'reward': reward
            }
        )

        epochs += 1
    total_reward += sum([i['reward'] for i in frame_set])
    total_successes += successes
    total_epochs += epochs
    frames.append(frame_set)

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average successes per episode: {total_successes / episodes}")
print(f"Average reward per episode: {total_reward / episodes}")

Results after 1000 episodes:
Average timesteps per episode: 5.466
Average successes per episode: 0.164
Average reward per episode: -1.182


In [27]:
frames[0]

[{'state': {'previous': {'last_action': 0, 'last_response': 0},
   'me_attributes': {'confidence': 0},
   'partner_attributes': {'interests': {'cars': 3,
     'sports': 2,
     'literature': 0,
     'history': 1,
     'machine learning and artificial intelligence': 1},
    'attraction': 2,
    'romance': 0}},
  'action': 3,
  'reward': -1},
 {'state': {'previous': {'last_action': 3, 'last_response': -1},
   'me_attributes': {'confidence': 0},
   'partner_attributes': {'interests': {'cars': 3,
     'sports': 2,
     'literature': 0,
     'history': 1,
     'machine learning and artificial intelligence': 1},
    'attraction': 1,
    'romance': 0}},
  'action': 6,
  'reward': -1},
 {'state': {'previous': {'last_action': 6, 'last_response': 0},
   'me_attributes': {'confidence': 1},
   'partner_attributes': {'interests': {'cars': 3,
     'sports': 2,
     'literature': 0,
     'history': 1,
     'machine learning and artificial intelligence': 1},
    'attraction': 1,
    'romance': 0}},
  

In [ ]:
state, reward, done